In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
import estimate_critical_q as est_qc
import estimate_exponents as est_exps
import tools

## Notebook parameters

In [ ]:
db_path = 'results_databases/bubble_filtering__8_neighbors_squared_network__MXU.csv'

In [ ]:
# Read the physical quantities database
full_phys_quant = pd.read_csv(db_path, index_col=0)

In [ ]:
# Values of the visibility parameter
#
vs = full_phys_quant.v.unique()[1:]

#### .
<br>
#### Go to the cell <a href='#..'>after the plots</a>
<br>
<br>

In [ ]:
# Calculate the critical values for all values of visibility

critical_quantities = []

for v in vs:
    
    _ = []
    
    # Select the data corresponding to the given visibility v.
    phys_quant = full_phys_quant[full_phys_quant.v == v]
    
    # Get the number of sites
    N = full_phys_quant.N.unique()

    # Set the index to N and q
    phys_quant.set_index(['N','q'], inplace=True)
    
    # Estimate the critical q
    qc, qc_err, _1, _2 = est_qc.estimate(N, phys_quant, do_plot=True)
    
    #qc, qc_err = tools.express_measure_with_error(qc, qc_err)
    
    _.extend([qc, qc_err])
    
    # Estimate the critical exponents
    for x in ['beta_nu', 'gamma_nu', 'inv_nu']:
    
        exp, exp_err, y_til_0, y_til_0_err, reg_rsquared = est_exps.estimate_exponent(
                                    x, qc, N, phys_quant, do_plot=True)
        
        #exp, exp_err = tools.express_measure_with_error(exp, exp_err)
        #y_til_0, y_til_0_err = tools.express_measure_with_error(y_til_0, y_til_0_err)
        
        _.extend([exp, exp_err, y_til_0, y_til_0_err, reg_rsquared])
        
        
        if x == 'beta_nu':
            beta_nu = exp
            beta_nu_err = exp_err
            
        if x == 'gamma_nu':
            gamma_nu = exp
            gamma_nu_err = exp_err
            
        if x == 'inv_nu':
            inv_nu = exp
    
    # Estimate the effective dimension
    D, D_err = est_exps.estimate_D(beta_nu, beta_nu_err, gamma_nu, gamma_nu_err)
    
    #D, D_err = tools.express_measure_with_error(D, D_err)
        
    _.extend([D, D_err])
    
    # Do data collapse plots
    est_exps.data_collapse(qc, beta_nu, gamma_nu, inv_nu, N, phys_quant, 
                           plot_title = 'Visibility v = %.2f' % v)
    
    print('\n\n\n\n\n\n\n\n')
    print('============================================================================')
    print('\n\n\n\n\n\n\n\n')
    
    critical_quantities.append(_)
    
    
    
# Create an data frame to store the critical quantities 
# for each value of the visibility parameter.
cols = ['qc', 'qc_err', 
           'beta_nu', 'beta_nu_err', 'M_til_0', 'M_til_0_err', 'beta_nu_R2',
           'gamma_nu', 'gamma_nu_err', 'X_til_0', 'X_til_0_err', 'gamma_nu_R2',
           'inv_nu', 'inv_nu_err', 'dU_til_0', 'dU_til_0_err', 'inv_nu_R2',
           'D', 'D_err'
          ]

critical_quantities = pd.DataFrame(columns=cols, data=critical_quantities, index=vs)

#### ..
<br>
#### Go to the cell <a href='#.'>before the plots</a>
<br>
<br>

In [ ]:
def plot_v_dep(var, var_err):
    plt.figure(figsize=(8,6))

    critical_quantities[var].plot(lw=0.5, color='black')
    inf_l = critical_quantities[var] - critical_quantities[var_err]
    sup_l = critical_quantities[var] + critical_quantities[var_err]
    plt.fill_between(critical_quantities.index, inf_l, sup_l, alpha=0.5, color='orange')

    plt.xlabel('v', fontsize=16)
    plt.ylabel(var, fontsize=16)

In [ ]:
plot_v_dep('qc', 'qc_err')

In [ ]:
plot_v_dep('beta_nu', 'beta_nu_err')
plt.ylim(0,1);

In [ ]:
plot_v_dep('M_til_0', 'M_til_0_err')
plt.ylim(0,2);

In [ ]:
plot_v_dep('gamma_nu', 'gamma_nu_err')
plt.ylim(0,1.2);

In [ ]:
plot_v_dep('X_til_0', 'X_til_0_err')
plt.ylim(0,1);

In [ ]:
plot_v_dep('inv_nu', 'inv_nu_err')
plt.ylim(0, 2);

In [ ]:
plot_v_dep('dU_til_0', 'dU_til_0_err')

In [ ]:
plot_v_dep('D', 'D_err')
plt.ylim(0,2);

In [ ]:
critical_quantities

In [ ]:
critical_quantities.beta_nu_R2.plot()
plt.title(r'$\beta/\nu$', fontsize=18)
plt.xlabel(r'$v$', fontsize=16)
plt.ylabel(r'$R^2$', fontsize=16)
plt.ylim(-0.05,1.05);

In [ ]:
critical_quantities.gamma_nu_R2.plot()
plt.title(r'$\gamma/\nu$', fontsize=18)
plt.xlabel(r'$v$', fontsize=16)
plt.ylabel(r'$R^2$', fontsize=16)
plt.ylim(-0.05,1.05);

In [ ]:
critical_quantities.inv_nu_R2.plot()
plt.title(r'$1/\nu$', fontsize=18)
plt.xlabel(r'$v$', fontsize=16)
plt.ylabel(r'$R^2$', fontsize=16)
plt.ylim(-0.05,1.05);